# Notes

* generate data from
  * isophonics functions
  * salami functions
  * l2 norm
* samples:
  * features = patches of cqt +- 5 frames on either side
  * positives = drawn from pairs of beats within the same segment
  * negatives = pairs of beats crossing a segment boundary
* model evaluation
  * serra method or laplacian

In [1]:
import numpy as np
import librosa
import cPickle as pickle
import numba
import os
from joblib import Parallel, delayed

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt

In [3]:
def symstack(X, n_steps=3, delay=1, **kwargs):
    '''Symmetric history stacking.
    
    like librosa.feature.stack_memory, but IN THE FUTURE!!!
    '''
    rpad = n_steps * delay
    Xpad = np.pad(X,
                  [(0, 0), (0, rpad)],
                  **kwargs)
    
    Xstack = librosa.feature.stack_memory(Xpad,
                                          n_steps=2 * n_steps + 1,
                                          delay=delay,
                                          **kwargs)
    
    return Xstack[:, rpad:]

In [4]:
def load_feature_data(file_name, n_steps=5):
    
    fname = '../data/features/{}.pk'.format(file_name)
    
    data = pickle.load(open(fname, 'r'))
    
    return symstack(data['cqgram'].astype(np.float32),
                    n_steps=n_steps,
                    mode='edge'), data['intframes']

In [5]:
@numba.jit
def make_training_data(max_samples=4,
                       paths=None,
                       file_name=None,
                       thresholds=None,
                       labels=None,
                       intervals=None,
                       fmeasures=None,
                       scores=None):
    
    # intframes gives the section->cqt frame alignments
    X_raw, intframes = load_feature_data(file_name)
    
    #   add observations for each (i, i+1)
    #   set the label to +1 unless i+1 is in intervals[:, 0]
    
    ex_x = np.abs(X_raw[:, :-1] - X_raw[:, 1:]).T
    ex_y = np.ones(len(ex_x), dtype=np.int32)
    
    negatives = intframes[:-1, 1] - 1
    ex_y[negatives] = 0
    
    return ex_x, ex_y

In [6]:
import os
def process_file(file_scores, max_samples=8):
    
    if file_scores['paths']:
        X, Y = make_training_data(max_samples=max_samples,
                                  **file_scores)
        outfile = '../data/labeled_features_local/{}_beats.npz'.format(os.path.basename(file_scores['file_name']))
        np.savez(outfile, X=X, Y=Y)

In [10]:
all_data = pickle.load(open('../data/scores_datasetESALAMI_levelEfunction_distEL2.pk'))
#all_data = pickle.load(open('../data/scores_datasetEIsophonics_levelEfunction_distEL2.pk'))

In [11]:
[(x, y['file_name']) for (x,y) in enumerate(all_data['file_scores'])]

[(0, 'SALAMI_396.mp3'),
 (1, 'SALAMI_822.mp3'),
 (2, 'SALAMI_748.mp3'),
 (3, 'SALAMI_564.mp3'),
 (4, 'SALAMI_584.mp3'),
 (5, 'SALAMI_550.mp3'),
 (6, 'SALAMI_1028.mp3'),
 (7, 'SALAMI_1300.mp3'),
 (8, 'SALAMI_248.mp3'),
 (9, 'SALAMI_960.mp3'),
 (10, 'SALAMI_1562.mp3'),
 (11, 'SALAMI_486.mp3'),
 (12, 'SALAMI_1180.mp3'),
 (13, 'SALAMI_990.mp3'),
 (14, 'SALAMI_1462.mp3'),
 (15, 'SALAMI_304.mp3'),
 (16, 'SALAMI_40.mp3'),
 (17, 'SALAMI_356.mp3'),
 (18, 'SALAMI_540.mp3'),
 (19, 'SALAMI_634.mp3'),
 (20, 'SALAMI_970.mp3'),
 (21, 'SALAMI_362.mp3'),
 (22, 'SALAMI_1394.mp3'),
 (23, 'SALAMI_772.mp3'),
 (24, 'SALAMI_842.mp3'),
 (25, 'SALAMI_866.mp3'),
 (26, 'SALAMI_12.mp3'),
 (27, 'SALAMI_1390.mp3'),
 (28, 'SALAMI_892.mp3'),
 (29, 'SALAMI_1258.mp3'),
 (30, 'SALAMI_1314.mp3'),
 (31, 'SALAMI_1338.mp3'),
 (32, 'SALAMI_986.mp3'),
 (33, 'SALAMI_1216.mp3'),
 (34, 'SALAMI_1012.mp3'),
 (35, 'SALAMI_264.mp3'),
 (36, 'SALAMI_1406.mp3'),
 (37, 'SALAMI_18.mp3'),
 (38, 'SALAMI_1092.mp3'),
 (39, 'SALAMI_1426.mp3')

In [12]:
Parallel(n_jobs=4, verbose=20)(delayed(process_file)(data) for data in all_data['file_scores']);

[Parallel(n_jobs=4)]: Done   1 out of 769 | elapsed:    0.4s remaining:  4.6min
[Parallel(n_jobs=4)]: Done  30 out of 769 | elapsed:    0.9s remaining:   21.0s
[Parallel(n_jobs=4)]: Done  69 out of 769 | elapsed:    1.7s remaining:   17.1s
[Parallel(n_jobs=4)]: Done 108 out of 769 | elapsed:    2.8s remaining:   17.3s
[Parallel(n_jobs=4)]: Done 147 out of 769 | elapsed:    4.0s remaining:   17.0s
[Parallel(n_jobs=4)]: Done 186 out of 769 | elapsed:    4.7s remaining:   14.9s
[Parallel(n_jobs=4)]: Done 225 out of 769 | elapsed:    5.4s remaining:   13.2s
[Parallel(n_jobs=4)]: Done 264 out of 769 | elapsed:    6.9s remaining:   13.1s
[Parallel(n_jobs=4)]: Done 303 out of 769 | elapsed:    8.0s remaining:   12.3s
[Parallel(n_jobs=4)]: Done 342 out of 769 | elapsed:   10.0s remaining:   12.4s
[Parallel(n_jobs=4)]: Done 381 out of 769 | elapsed:   10.8s remaining:   11.0s
[Parallel(n_jobs=4)]: Done 420 out of 769 | elapsed:   12.1s remaining:   10.1s
[Parallel(n_jobs=4)]: Done 459 out of 76